# Об'єктно орієнтовване програмування
---
Парадигма ООП містить у собі 4 ключові принципи:

1. Інкапсуляція
2. Успадкування
3. Поліморфізм
4. Абстракція

У цьому ноутбуці розглянуто перші два з них.

## Інкапсуляція

Інкапсуляція - один з основних механізмів ООП, який забезпечує виконання наступного правила:
> Доступ напряму до стану об'єкта **заборонений**. Маніпуляція об'єктом відбувається за допомогою спеціально розробленого інтерфейсу



#### Чому це важливо?
Цей механізм забезпечує низку корисних властивостей, ось деякі з них:

 - Запобігання неправильного використання
 - Гнучкість реалізації
 - Конфіденційність даних

##### Запобігання неправильного використання

Оскільки маніпулювати об'єктом можна лише через чітко вказаний інтерфейс, розробник маже контролювати звертання до конкретних атрибутів у будь-який момент часу, таким чином унеможливлюється ситуація невалідних значень в атрибутах, неправильних викликів методів, тощо.

##### *Приклад*
Реалізація розумного списку, який при звертанні за будь-яким індексом повертає елемент, позиція якого дорівнює модулю вказаного числа за довжиною списку (лишок при діленні на довжину списку, фактично дозволяє циклічно рухатися по списку).


In [1]:
class SmartList:
  def __init__(self, initial_list: list = []):
    self.lst = initial_list
    self.length = len(initial_list)

  def value(self, idx: int):
    return self.lst[idx % self.length]

Якщо ж ми створимо об'єкт даного класу та спробуємо змінити лише одне з двох полів, ми отримаємо невалідний об'єкт, який буде видавати неправильний результат.

In [2]:
smart_list = SmartList([1, 2, 3])
print(f"Third element in smart list is {smart_list.value(3)}.")
smart_list.lst += [4]
print(f"After inserting one more element, the new list is {smart_list.lst}.\n" 
      f"The third element in smart list is {smart_list.value(3)}.\n"
      f"However it should be {smart_list.lst[3]}.")

Third element in smart list is 1.
After inserting one more element, the new list is [1, 2, 3, 4].
The third element in smart list is 1.
However it should be 4.


##### Гнучкість реалізації

Інтерфейс за допомогою якого маніпулюють об'єктом не залежить від реалізації, а отже розробник може змінювати логіку яка використовується в коді, допоки не змінюється інтерфейс

##### *Приклад*
Знаходження координат країни за її назвою. Інтерфейс функції:

`get_coordinates(country_name: str) -> list[float]`

In [11]:
# Реалізація №1
def get_coordinates(country_name: str) -> list:
  from geopy.geocoders import Nominatim
  geolocator = Nominatim(user_agent="specify_your_app_name_here")
  location = geolocator.geocode(country_name)
  return [location.latitude, location.longitude]

In [12]:
# Реалізація №2
def get_coordinates(country_name: str) -> list:
  import json
  with open("country_coords.json", 'r', encoding='utf-8') as country_codes:
    data = json.load(country_codes)
  return data[country_name]

Часто ви зможете лише здогадуватись яка саме реалізація у того чи іншого метода чи функції, а час від часу реалізація може змінюватися, але при цьому спосіб використання цієї функції завжди має бути сталим

##### Конфіденційність даних

Заради коректної роботи методів часто необхідне використання тих чи інших конфіденційних даних, доступ до яких має бути обмеженим для користувача, що забезпечується відсутністю способів звертань до них за допомогою інтерфейсу

##### *Приклад*
Ключі для використання API методів. Оскільки їх використання може бути обмеженим, або інколи навіть тарифікуватись, їх секретність є критично важливою, а отже, до змінних в яких зберігаються такі ключі не має бути доступу в користувача

#### Інкапсуляція в python

За реалізацію інкапсуляції в різних мовах програмування відповідають певні ключові слова, які явно вказують можливість доступу до конкретного атрубуту чи методу. Зазвичай використовуються наступні:

 - **public** - доступ дозволено для усіх
 - **protected** - доступ захищено, звертання дозволене лише для дочірних класів
 - **private** - доступ заборонено для усіх

У python можливість задати рівень доступу до аргументу чи функції реалізований в інший спосіб:

 - Будь-який атрибут/метод по замовчування вважається публічним (*public*)
 - Метод/атрибут, чия назва починається з символа `_` вважається захищенним (*protected*)
 - Метод/атрибут, чия назва починається з символів `__` вважається приватним (*private*)

**Варто зауважити:**

 1. Виключенням з цього правила є методи які починаються та закінчуються на `__`, ці методи є **публічними**
 2. Інтерпретатор python **не перевіряє** доступ до захищених елементів. Такий синтаксис є домовленістю, а не правилом, та слугує підказкою програмісту

In [5]:
class VerySmartList:
  def __init__(self, initial_list: list = []):
    self.__lst = initial_list
    self.__length = len(initial_list)

  def value(self, idx: int):
    return self.__lst[self._actual_index(idx)]

  def _actual_index(self, idx: int):
    return idx % self.__length

  def __add_element__(self, new_element):
    self.__lst.append(new_element)
    self.__length += 1

In [6]:
very_smart_list = VerySmartList([1, 2, 3])
print(very_smart_list.value(3))            # Метод value є публічним, а отже його можна викликати
print(very_smart_list._actual_index(3))   # Метод _actual_index є захищеним, його виклик не призведе до помилки
print(very_smart_list.__add_element__(1))   # Метод __add_element__ є публічним, оскільки він починається та закінчується на __
# print(very_smart_list.__lst)             # Атрибут __lst є приватним, а отже спроба його отримати викличе помилку

1
0
None


#### Getter, setter, property

Часто певні атрибути класу мають задовільняти якісь умови, наприклад: ім'я має бути рядком, але не пустим, вік має бути цілим числом, але не від'ємним, тощо. Робити такі атрибути публічними може бути поганою ідеєю, адже тоді їм можна встановити будь-яке значення, що може призвести до помилок виконання, але й приватними їх не зробиш, оскільки важливо мати можливість змінювати значення атрибуту 

Цю проблему зазвичай вирішують створюючи **спеціальні методи**, так звані гетери та сетери, з необхідними перевіркою  залишаючи при цьому атрибут приватним

In [7]:
class GetterSetterList:
  def __init__(self, initial_list: list = []):
    self.__lst = initial_list
    self.__length = len(initial_list)

  def value(self, idx: int):
    return self.__lst[self._actual_index(idx)]

  def _actual_index(self, idx: int):
    return idx % self.__length

  def __add_element__(self, new_element):
    self.__lst.append(new_element)
    self.__length += 1

  def get_list(self):               # Getter
    return self.__lst
  
  def get_length(self):             # Getter
    return self.__length

  def set_list(self, new_list):     # Settter
    if not isinstance(new_list, list) or not new_list:
      # Виконується тільки якщо аргумент методу не список, або порожній список
      raise ValueError("Only non-emty lists are suported")
    # Виконується тільки якщо аргумент методу не порожній список
    self.__lst = new_list
    self.__length = len(new_list)

За допомогою так званих гетерів (`get_list`, `get_length`) можна дізнаватися значення приватних атрибутів, а за допомогою сетерів (`set_list`) можна задавати значення атрибутам, враховуючи при цьому всі необхідні обмеження

**Важливо**: викликаючи гетер, програміст очікує дізнатися про поточний стан об'єкта, а тому зміна значень атрибутів в гетері - дуже погана практика, яка призводить до дивних помилок

In [8]:
getter_setter_list = GetterSetterList([1, 2, 3])
print(f"Initialized list is {getter_setter_list.get_list()}")
print(f"Length of the initialized list is {getter_setter_list.get_length()}")
getter_setter_list.set_list([4, 5, 6])
print(f"Newly setted list is {getter_setter_list.get_list()}\n")

try:
  print(f"I can reach the list itself, it is {getter_setter_list.__lst}")
except AttributeError as error:
  print(f"I still can not get the list itself:\n{repr(error)}\n")

try:
  getter_setter_list.set_list(123)
  print(f"I managed to set a number to a list: {getter_setter_list.get_list()}")
except ValueError as error:
  print(f"I can not set a number as a list:\n{repr(error)}")

Initialized list is [1, 2, 3]
Length of the initialized list is 3
Newly setted list is [4, 5, 6]

I still can not get the list itself:
AttributeError("'GetterSetterList' object has no attribute '__lst'")

I can not set a number as a list:
ValueError('Only non-emty lists are suported')


Використання дивних функцій для встановлення значень у певний атрибут інколи може заплутати, адже зазвичай це виконується за допомогою звичайного присвоєння. Для цього у класі можна створити так звану **властивість** (property), доступатись до якої можна аналогічним способом, як і до атрибутів. 
На відміну від звичайних атрибутів класу при читанні та запису у властивість буде викликатися відповідний метод, вказаний при створенні властивості:

`property_name = property(getter, setter, deleter, docstring)`

In [12]:
class PropertyList:
  def __init__(self, initial_list: list = []):
    self.__lst = initial_list
    self.__length = len(initial_list)

  def value(self, idx: int):
    return self.__lst[self._actual_index(idx)]

  def _actual_index(self, idx: int):
    return idx % self.__length

  def __add_element__(self, new_element):
    self.__lst.append(new_element)
    self.__length += 1

  def __get_list(self):                 # Getter
    return self.__lst

  def __set_list(self, new_list):       # Setter
    if not isinstance(new_list, list) or not new_list:
      # Виконується тільки якщо аргумент методу не список, або порожній список
      raise ValueError("Only non-emty lists are suported")
    # Виконується тільки якщо аргумент методу не порожній список
    self.__lst = new_list
    self.__length = len(new_list)

  def __del_list(self):                 # Deleter
    self.__lst = []
    self.__length = 0

  lst = property(__get_list, __set_list, __del_list, "List property")

**Варто зауважити, що самі методи варто робити приватними, щоб не засмічувати `__dict__`** 

In [13]:
property_list = PropertyList([1, 2, 3])
print(f"Initial value of the list is {property_list.lst}")
property_list.lst = [4, 5, 6]
print(f"New value of the list is {property_list.lst}")
del property_list.lst
print(f"Value of the list after deleting is {property_list.lst}")

Initial value of the list is [1, 2, 3]
New value of the list is [4, 5, 6]
Value of the list after deleting is []


Існує також й інший спосіб створити властивість класу, за допомогою **декораторів**, при цьому назва властивості співпадатиме з назвами усіх використаних функцій

In [25]:
class DecoratedPropertyList:
  def __init__(self, initial_list: list = []):
    self.__lst = initial_list
    self.__length = len(initial_list)

  def value(self, idx: int):
    return self.__lst[self._actual_index(idx)]

  def _actual_index(self, idx: int):
    return idx % self.__length

  def __add_element__(self, new_element):
    self.__lst.append(new_element)
    self.__length += 1

  @property
  def lst(self):                 # Getter
    return self.__lst

  @lst.setter
  def lst(self, new_list):       # Setter
    if not isinstance(new_list, list) or not new_list:
      # Виконується тільки якщо аргумент методу не список, або порожній список
      raise ValueError("Only non-emty lists are suported")
    # Виконується тільки якщо аргумент методу не порожній список
    self.__lst = new_list
    self.__length = len(new_list)

  @lst.deleter
  def lst(self):                 # Deleter
    self.__lst = []
    self.__length = 0

In [26]:
property_list = DecoratedPropertyList([1, 2, 3])
print(f"Initial value of the list is {property_list.lst}")
property_list.lst = [4, 5, 6]
print(f"New value of the list is {property_list.lst}")
del property_list.lst
print(f"Value of the list after deleting is {property_list.lst}")


Initial value of the list is [1, 2, 3]
New value of the list is [4, 5, 6]
Value of the list after deleting is []


## Успадкування

Успадкування - один з основних механізмів ООП, який надає класу можливість використовувати код іншого, **доповнюючи** його власними деталями реалізації

#### Чому це важливо?
Використання успадкування у вашому коді не змінить спосіб його використання в інших місцях, проте може значно спростити розробку. Використання успадкування дозволяє:

 - Уникнути дублювання коду, перевикориставши вже написаний
 - Використовувати бібліотеки, розширюючи потрібні класи в разі необхіддності
 - Ліпше зрозуміти логіку проєкту

##### Перевикористання власного коду
За необхідності реалізувати клас, який за своїм змістом є підмножиною іншого, вже реалізованого класу, скопіювати необхідний функціонал буде поганою ідеєю. Крщим рішенням буде успадкувати новостворений клас від вже існуючого

##### *Приклад*
Окрім можливості отримати значення за будь-яким індексом, розумний список має мати метод, який поверне мінімальне значення

In [14]:
class MinSmartList(PropertyList):
  def get_min(self):
    return min(self.lst)

Такий клас матиме увесь функціонал, який присутній у класі `PropertyList`, а також метод для знаходження мінімуму

In [15]:
min_smart_list = MinSmartList([1, 2, 3])
print(f"Initialized list is {min_smart_list.lst}")
print(f"Thrd element in the list is {min_smart_list.value(3)}")
print(f"Minimum value in the list is {min_smart_list.get_min()}")

Initialized list is [1, 2, 3]
Thrd element in the list is 1
Minimum value in the list is 1


##### Використання бібліотек
Інколи класи створені іншими розробниками часто не повністю відповідають вимогам конкретної програми, оскільки в них бракує певного специфічного функціоналу. У такому випадку варто створити власний клас, який буде успадковувати існуючий, та матиме окримо прописані, необхідні саме конкретній програмі властивості

##### *Приклад*
Реалізація списку, доступитися до елементів якого можна за допомогою будь-якого числа з використанням успадкування може вигладати отак

In [27]:
class InheritedList(list):
  def value(self, idx: int):
    return self[idx % len(self)]

Оскільки новий клас є нащадком `list`, він має повний функціонал цього класу, чого не мали класи неаписані вище

In [28]:
inherited_list = InheritedList([1, 2, 3, 4, 5, 6])
print(f"Initialized list is {inherited_list}")
print(f"Sixth element in the list id {inherited_list.value(6)}")
print(f"Also I can slice this list: inherited_list[1:5:2] = {inherited_list[1:5:2]}")

Initialized list is [1, 2, 3, 4, 5, 6]
Sixth element in the list id 1
Also I can slice this list: inherited_list[1:5:2] = [2, 4]


#### Успадкування в python
Успадковування в python має наступний синтаксис:

`class Child(Parent1, Parent2, ...)`,

при цьому клас `Child` називають **дочірним** або підкласом, а класи `Parent1`, `Parent2` називають **батьківськими** або суперкласами

Для перевірки чи є об'єкт екземпляром класу використовується функція **`isinstance`**, а для перевірки, чи є клас підкласом іншого класу використовується функція **`issubclass`**

In [ ]:
class A:
  pass
class B(A):
  pass

In [ ]:
a_object = A()
b_object = B()
print(f"a_object is an instance of class A: {isinstance(a_object, A)}")
print(f"b_object is an instance of class A: {isinstance(b_object, A)}")
print(f"a_object is not an instance of class B: {isinstance(a_object, B)}\n")
print(f"B is a subclass of class A: {issubclass(B, A)}")
print(f"A is not a subclass of class B: {issubclass(A, B)}")

a_object is an instance of class A: True
b_object is an instance of class A: True
a_object is not an instance of class B: False

B is a subclass of class A: True
A is not a subclass of class B: False


У python будь-який клас є підкласом спеціального класу **`object`**, а тому наступні оголошення є еквівалентними:

`class SomeClass(object)` ≡ `class SomeClass`

In [ ]:
print(f"A is subclass of class object: {issubclass(A, object)}")
print(f"123 is instance of class object: {isinstance(123,)}")

A is subclass of class object: True
123 is instance of class object: True


#### Простір імен
Кожен об'єкт та кожен клас у python є простором імен, звичайним списком атрибутів, до яких можна доступитися. При успадкуванні класу `Parent` класом `Child`, до методів та атрибутів класу `Child` дописується простір імен класу `Parent`, що означає можливість викликати метод класу `Parent` з об'єкту класу `Child`

При кожному звертанні до методів/атрибутів, інтерпретатор запускає пошук вказаного імені у просторі імен об'єкту, та звертається до першого знайденого

Для дослідження просторів імен можна використати функцію `dir`, або атрибут `__dict__`. Порядок дослідження просторів імен для 
класу можна дізнатися за допомогою методу `mro`

In [ ]:
class A:
  def method_from_a(self):
    pass
class B(A):
  def method_from_b(self):
    pass
class C(B):
  def method_from_c(slef):
    pass

In [ ]:
print(f"Name space for class A is {dir(A)}")
print(f"Name space for class B is {dir(B)}")
print(f"Name space for class C is {dir(C)}\n")
print(f"The order of name spaces to search in for class C is {C.mro()}")

Name space for class A is ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'method_from_a']
Name space for class B is ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'method_from_a', 'method_from_b']
Name space for class C is ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__',

Пам'ятаємо, що клас `A` успадковується від `object`, звідси і купа дивних методів у кожному з просторів імен, усі воони - елементи класу `object`

#### Super
Часто для реаліацій підкласів буває корисно скористатися методом чи атрибутом суперкласу. Завдяки наявності простору імен суперкласу у просторі підкласу, зазвичай це можливо зробити дуже легко: `obj.attr`, але проблеми виникають тоді, коли існують методи/атрибути з однаковими іменами (наприклад  `__init__`). Викликати такі методи можна у 2 способи:

`SuperClass.method(self, arg1, arg2, ...)`

`super().method(arg1, arg2, ...)`

Функція `super()` повертає тимчасовий об'єкт суперкласу, а отже, в його просторі імен існують лише методи суперкласу, один з яких ми й викликаємо. Варто зауважити, що використання функції `super()` є більш універсальним за виклик функції з класу

In [ ]:
class A:
  def method(self):
    print("Method from class A")
class B(A):
  def nethod(self):
    print("Method from class B")
  def self_call(self):
    print("Using self.nethod()...")
    self.nethod()
  def super_call(self):
    print("Using super().method()...")
    super().method()


B().self_call()
B().super_call()

Using self.nethod()...
Method from class B
Using super().method()...
Method from class A


#### Diamond problem
У python, на відміну від багатоьлх інших мов програмування, є можливість успадковуватися від декількох суперкласів одночасно, через що виникають певні проблеми

> Який атрибут буде використовувати інтерпретатор Python, якщо в графі успадкування в суперкласах визначені атрибути з однаковими іменами?

Для кожного об'єкта в python визначений список з просторів імен в яких необхідно шукати вказані еоементи. Дізнатися у якому саме порядку та у яких простороах інтерпретатор буде шукати можна за допомогою методу `mro`

В разі множинного успадкування, пошук імен у суперкласах буде відбуватись у порядку, в якому вони були вказаані **при оголошені** класу.

In [29]:
class A:
  def __init__(self):
    super().__init__()
    self.attr = "attribute from class A"
class B:
  def __init__(self):
    super().__init__()
    self.attr = "attribute from class B"
class AB(A, B):
  def __init__(self):
    super().__init__()
    print(f"Attribute attr from class AB(A, B) is {self.attr}")
class BA(B, A):
  def __init__(self):
    super().__init__()
    print(f"Attribute attr from class BA(B, A) is {self.attr}")

In [30]:
print(f"Method resolution order for AB is {AB.mro()}")
print(f"Method resolution order for BA is {BA.mro()}")

Method resolution order for AB is [<class '__main__.AB'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>]
Method resolution order for BA is [<class '__main__.BA'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>]


**Важливо зауважити**: виклик методу `super()` повертає екзкмпляр класу який іде наступним у цьому списку після поточного, і це не завжди буде суперкласом

In [31]:
class Base:
  def __init__(self):
    print("Base init")

class A(Base):
  def __init__(self):
    print("A init")
    super().__init__() # B.__init__ !!! 

class B(Base):
  def __init__(self):
    print("B init")
    super().__init__() # Base.__init__
    
class C(A, B):
  def __init__(self):
    print("C init")
    super().__init__() # A.__init__

In [ ]:
C()
C.mro()

C init
A init
B init
Base init


[__main__.C, __main__.A, __main__.B, __main__.Base, object]